# Analysis Walkthrough

## Determine all of the parameters

### Specify the locations of the files for X_train, X_test, and y_train. Also, there is a file that contains information about the individual stations that can be useful for models that learn for each station.

In [3]:
import os
os.chdir('..')

In [4]:
Xtrain_dir = 'solar/data/kaggle_solar/train/'
Xtest_dir = 'solar/data/kaggle_solar/test'
ytrain_file = 'solar/data/kaggle_solar/train.csv'
station_file = 'solar/data/kaggle_solar/station_info.csv'

### Import the various files. This is mostly done so that any file updates during testing are carried over to this notebook.

In [5]:
import solar.wrangle.wrangle
import solar.wrangle.subset
import solar.wrangle.engineer
import solar.analyze.model
import solar.report.submission
import numpy as np

### Set the parameters that will be used to set up the data. There are some parameters that determine the size and shape of the data but have effects other than setting up feature columns. This includes the dates that are included for testing and training, the stations considered, and whether to have X values correspond to a date or to a specific date/station combination.

In [6]:
# Choose up to 98 stations; not specifying a station means to use all that fall within the given lats and longs. If the
# parameter 'all' is given, then it will use all stations no matter the provided lats and longs
station = ['ACME', 'BEAV', 'ADAX']
#station = ['all']

# Determine which dates will be used to train the model. No specified date means use the entire set from 1994-01-01 
# until 2007-12-31.
#train_dates = ['1994-01-01','2007-12-31']
train_dates = ['2005-11-30','2006-01-02']
# Determine the test X values to produce. There is no practical purpose to use fewer than all of the points other than
# for testing. Again, not choosing a date will use 2008-01-01 through 2012-11-30.
test_dates = ['2008-12-29', '2009-02-04']
#test_dates = ['all']
#test_dates = []
#train_dates = []
#test_dates = []

# The last parameter that is not specifically involved in feature selection in the layout to be used for training
# I have switched to almost always training for each individual station rather than having a single row for a date.
# However, I am still not beating the benchmark, and the file would grow too large to have the benchmark laid out
# with a row for each station, so I'll keep the switch. True means that each station has a row (5113 dates X 98
# stations to train the data). False means that there are 5113 rows that are being used to train the data.
station_layout = True

### First, just duplicate the functionality of the basic grid analysis

In [30]:
# Use all variables
var = ['uswrf_sfc', 'dswrf_sfc']
#var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model = [0,7]

# Aggregate over all times
times = [18,21]

# Aggregate over ACME and BEAV
#station = ['ACME', 'BEAV']
#station = ['all']

default_grid = {'type':'relative', 'axes':{'var':var, 'models':model, 'times':times,
                                          'station':station}}
just_grid = [default_grid]


### Run data extraction

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

%prun input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, just_grid)

In [ ]:
#input_data[3]

### Run through the full analysis

In [ ]:
reload(solar.analyze.model)
import numpy as np
from solar.analyze.model import Model

from sklearn.linear_model import Ridge
from sklearn import metrics

error_formula = 'mean_absolute_error'
cv_splits = 10

model = Model.model(input_data[0:3], Ridge, {'alpha':np.logspace(-3,1,8,base=10)}, cv_splits, 
                    error_formula, normalize=True)

In [ ]:
reload(solar.report.submission)
from solar.report.submission import Submission

preds = Submission.make_submission_file(model, input_data[1], input_data[2], {'grid'})

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# here we set some aesthetic parameters so that all of our figures are nice and big
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
sns.set(style="white", context="talk")

In [ ]:
y_pred = model.predict(input_data[0])

In [ ]:
errors = abs(y_pred - input_data[1])/input_data[1]

In [ ]:
#sns.distplot(errors, rug=True)

In [ ]:
#input_data[0].shape

In [ ]:
#errors.shape

In [ ]:
#input_data[0][(errors > 0.8).values]

In [ ]:
#input_data[1][(errors > 0.8).values]

In [ ]:
#pd.DataFrame(y_pred, index=input_data[1].index, columns=input_data[1].columns)[(errors > 0.8).values]

In [ ]:
#max(errors.values)

In [ ]:
#unstacked = input_data[0].stack('time').stack('model').stack('variable').stack('lat_longs').reset_index('model').reset_index('time').reset_index('variable').reset_index('lat_longs')

In [ ]:
#dswrf = unstacked[(unstacked['lat_longs'] == 'NE') & (unstacked['variable'] == 'dswrf_sfc') & (unstacked['time'] == 21)
#         & (unstacked['model'] == 0)]['relative']

In [ ]:
#pd.DataFrame(dswrf).reset_index('station').rename(columns={'station':'location'}).unstack('location').set_index('location')

In [ ]:
#dswrf

In [ ]:
#pd.concat((errors,dswrf))

In [ ]:
#pd.concat([dswrf,errors])

In [ ]:
#sns.jointplot("total_solar","dswrf", data=drinks[(drinks.beer < 100) & (drinks.wine < 30)] , kind = "kde")

In [ ]:
#import netCDF4 as nc
#X = nc.Dataset('solar/data/kaggle_solar/train/dswrf_sfc_latlon_subset_19940101_20071231.nc','r+').variables.values()
#X[-1][0:10,0,2:3,:,:]

In [ ]:
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}
stat_feats = [stat_names]

### Next, we start to layout the features to include. The two most important (and complicated) are 'absolute', which just reports out the weather variables at specific GEFS, times, and models, and 'relative' which uses a grid to identify nearby GEFS for weather measurements based on the location of the station. The second option makes the most sense when using the station_layout above, but it will work with either layout.

In [ ]:
# A very simple model would just take the average value of all variables at all locations, using all models over the
# course of the day. Here, only the var parameter and one value of the model is expanded. 
# All of the other axes are aggregated using an aggregation function. In this case, the mean value. 
# This will provide a 15 aggregated columns for model 0 and 15 aggregated columns for the mean of models 1 though 10.
# In this case, setting station_layout to false would make the most sense because the measurements will be repeated
# for each station. However, for consistency in this walkthough, I will just keep it in the station_layout.

# Dimensions without aggregation

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model1 = [0]

# Dimensions with aggregation

# Aggregate over all other models (excluding model 0, which is used directly)
model2 = range(1,11)

# Aggregate over all times
times = ['all']

# Aggregate over all latitudes which surround Mesonet stations (exclude those that are outside of the main grid)
lats = range(33,38)

# Same as for lats
longs = range(257,267)

all_avgs = {'type':'absolute', 'full_axes':{'var':var, 'models':model1}, 
            'agg_axes':{'models':[model2,[np.mean]], 'times':[times, [np.mean, np.sum]], 'lats':[lats,[np.median]],
                        'longs':[longs,[np.median]]}}

avgs_feats = [all_avgs]

In [ ]:
# A similar example using a surrounding grid for each station. There are no lat or long options for this type of
# feature set

# Dimensions without aggregation

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model1 = [0]

# Dimensions with aggregation

# Aggregate over all other models (excluding model 0, which is used directly)
model2 = range(1,11)

# Aggregate over all times
times = ['all']

# Create a column for each member of the grid. All or nothing for gefs now. Could specify but currently see no need
# for it. We could also take an aggregate measure of the gefs (including interpolate). That doesn't work for the
# other dimensions

gefs = ['all']

grid_avgs = {'type':'relative', 'full_axes':{'var':var, 'models':model1, 'gefs':gefs}, 
            'agg_axes':{'models':[model2,[np.mean]], 'times':[times, [np.mean, np.sum]]}}

grid_feats = [grid_avgs]

In [ ]:
# A similar example using a surrounding grid for each station. Now, just average over the grid

# Dimensions without aggregation

# Use all variables
var = ['all']

# Keep model 0 (the default model) as a column for each of the variables (aggregated over other dimensions)
model1 = [0]

# Dimensions with aggregation

# Aggregate over all other models (excluding model 0, which is used directly)
model2 = range(1,11)

# Aggregate over all times
times = ['all']

# Create a column for each member of the grid. All or nothing for gefs now. Could specify but currently see no need
# for it. We could also take an aggregate measure of the gefs (including interpolate). That doesn't work for the
# other dimensions

gefs = ['all']

grid_avgs = {'type':'relative', 'full_axes':{'var':var, 'models':model1}, 
            'agg_axes':{'models':[model2,[np.mean]], 'times':[times, [np.mean, np.sum]], 'gefs':[gefs, [np.mean]]}}

gefs_mean_feats = [grid_avgs]

In [26]:
# This just uses the station_names as another feature
stat_names = {'type':'station_names'}
stat_feats = [stat_names]

In [18]:
frac_dist = {'type':'frac_dist'}
dist_feats = [frac_dist]

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

dist_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, dist_feats)

In [1]:
days_solstice = {'type':'days_from_solstice'}
days_cold = {'type':'days_from_coldest'}
days_feats = [days_solstice, days_cold]

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

dist_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, days_feats)

In [31]:
all_feats = [frac_dist, days_cold, days_solstice, stat_names, default_grid]

In [32]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

all_feats_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, all_feats)

In [33]:
all_feats_data[0]

(frac_dist, lat_dist)  (frac_dist, long_dist)  \
train_dates station                                                  
2005-11-30  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-01  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-02  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-03  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-04  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-05  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-06  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-07  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-08  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-09  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
...                                    ...                     ...   
2005-12-24  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-25  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-26  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-27  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-28  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-29  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-30  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2005-12-31  ACME                   0.80833                 0.97675   
            ADAX                   0.79851                 0.33091   
            BEAV                   0.80253                 0.46988   
2006-01-01  ACME               

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

stat_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, stat_feats)

In [ ]:
# if I am modifying code for any of these pythons
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

grid_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, just_grid)

In [ ]:
# test combination of station names and grid
reload(solar.wrangle.wrangle)
reload(solar.wrangle.subset)
reload(solar.wrangle.engineer)
from solar.wrangle.wrangle import SolarData

input_data = SolarData.load(Xtrain_dir, ytrain_file, Xtest_dir, station_file, \
                                  train_dates, test_dates, station, \
                                  station_layout, grid_and_stat_feats)

In [ ]:
stat_data = stat_data[0]

In [ ]:
len(grid_data.columns.levels)

In [ ]:
unstacked = grid_data.stack(range(0,grid_data.columns.nlevels))

In [ ]:
stat_data.columns.nlevels

In [ ]:
unstacked2 = stat_data.stack(range(0,stat_data.columns.nlevels))

In [ ]:
stat_data.columns[0]

In [ ]:
import pandas as pd
grid_data[stat_data.columns.names[0]]=stat_data.iloc[:,0]
combo = pd.concat([grid_data, stat_data], axis=1)

In [ ]:
#combo.stack()

In [ ]:
#import pandas as pd
#test = input_data[2]
#pd.get_dummies(test.stack('lat_longs').reset_index('lat_longs'),prefix='latlong', )
#test['station_name']
#pd.get_dummies(test['station_name'],prefix='stat')

In [ ]:
int(calendar.isleap(date1.year))

In [ ]:
import datetime
import calendar

dates = ['1994-01-01', '1994-01-02', '1996-01-04']
date1 = datetime.datetime.strptime('1996-12-21', '%Y-%m-%d')
date2 = datetime.date(date1.year, 12, 21)
date3 = datetime.date(date1.year, 12, 31)
abs_diff = abs((date1.timetuple().tm_yday) - (date2.timetuple().tm_yday))
if (abs_diff >= 183):
    final = date3.timetuple().tm_yday - abs_diff
else:
    final = abs_diff 
final


In [ ]:
((date1.timetuple().tm_yday) - (date2.timetuple().tm_yday))%183

In [ ]:
date2